In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# -------------------------------
# VIF 제거 함수 그대로 사용
# -------------------------------
def remove_high_vif_features(df, vif_threshold=10.0):
    """
    VIF를 계산하고 임계값보다 높은 VIF를 가진 변수를 반복적으로 제거합니다.
    df : 독립변수 DataFrame (종속변수 제외)
    """
    X = add_constant(df)

    print("다중공선성 변수 제거 프로세스 시작...")

    while True:
        features = X.columns.tolist()
        if "const" in features:
            features.remove("const")

        if len(features) <= 1:
            print("\n남은 변수가 1개 이하이므로 종료합니다.")
            break

        vif_data = pd.DataFrame()
        vif_data["feature"] = features
        vif_data["VIF"] = [
            variance_inflation_factor(X[features].values, i)
            for i in range(len(features))
        ]

        max_vif = vif_data["VIF"].max()

        if max_vif > vif_threshold:
            feature_to_remove = vif_data.loc[vif_data["VIF"].idxmax(), "feature"]
            X = X.drop(columns=[feature_to_remove])
            print(f"제거: {feature_to_remove} (VIF: {max_vif:.2f})")
        else:
            print(f"\n모든 변수의 VIF가 임계값 ({vif_threshold:.1f}) 이하입니다.")
            print(f"    - 최대 VIF: {max_vif:.2f}")
            break

    if "const" in X.columns:
        return X.drop(columns=["const"])
    return X


# -------------------------------
# 1) 데이터 불러오기
# -------------------------------
path = "../data/output_data/log.csv"
df = pd.read_csv(path)

# -------------------------------
# 2) 다중공선성 확인할 독립변수 X 만들기
#    - Date, 타깃컬럼(종가/목표값 등)은 제외
# -------------------------------
# 숫자형 컬럼만 선택
num_df = df.select_dtypes(include=[np.number]).copy()

# 타깃/제외하고 싶은 컬럼 이름들 (데이터에 맞게 필요하면 추가/삭제)
exclude_cols = [
    "y_target_log"
]
exclude_cols = [c for c in exclude_cols if c in num_df.columns]

X = num_df.drop(columns=exclude_cols)

# 분산 0인 상수 컬럼 제거 (VIF 계산 오류 방지)
X = X.loc[:, X.var() > 0]

print("원래 피처 수:", X.shape[1])

# -------------------------------
# 3) VIF 기반으로 다중공선성 높은 변수 제거
# -------------------------------
X_vif = remove_high_vif_features(X, vif_threshold=10.0)

print("\nVIF 제거 후 피처 수:", X_vif.shape[1])
print("남은 컬럼 목록:")
print(X_vif.columns.tolist())

# -------------------------------
# 4) 최종 VIF 표 한번 더 확인 (선택)
# -------------------------------
X_const = add_constant(X_vif)
features = [c for c in X_const.columns if c != "const"]

vif_table = pd.DataFrame({
    "feature": features,
    "VIF": [variance_inflation_factor(X_const[features].values, i)
            for i in range(len(features))]
}).sort_values("VIF", ascending=False)

print("\n최종 VIF 테이블:")
print(vif_table)


원래 피처 수: 42
다중공선성 변수 제거 프로세스 시작...
제거: PAYEMS (VIF: 21770153.37)
제거: SPS(달러) (VIF: 16233684.46)
제거: CPIAUCSL (VIF: 9365022.30)
제거: GC=F_Close (VIF: 7494266.22)
제거: High (VIF: 7476593.79)
제거: SHY_Close (VIF: 1639121.21)
제거: Close (VIF: 1297659.98)
제거: BPS(달러) (VIF: 1110585.40)
제거: 유동비율(%) (VIF: 305472.44)
제거: DX-Y.NYB_Close (VIF: 248867.25)
제거: ROIC(%) (VIF: 166979.18)
제거: Low (VIF: 89179.90)
제거: 부채비율(%) (VIF: 73150.21)
제거: ROA(%) (VIF: 61768.67)
제거: TLT_Close (VIF: 25458.84)
제거: GLD_Close (VIF: 14485.14)
제거: UNRATE (VIF: 9902.69)
제거: Volume (VIF: 9476.98)
제거: EPS(달러) (VIF: 6801.56)
제거: WTI_Spot (VIF: 4742.81)
제거: TLT_Volume (VIF: 4415.93)
제거: 이자보상
배수(배) (VIF: 3726.07)
제거: GLD_Volume (VIF: 2411.52)
제거: PCR(배) (VIF: 2081.60)
제거: SHY_Volume (VIF: 1232.59)
제거: PSR(배) (VIF: 1203.32)
제거: 기준금리 (VIF: 455.30)
제거: DPS(달러) (VIF: 353.52)
제거: ^VIX_Close (VIF: 246.00)
제거: TNX (%) (VIF: 193.95)
제거: PBR(배) (VIF: 143.84)
제거: ROE(%) (VIF: 60.67)
제거: Volatility (VIF: 34.18)
제거: CPS(달러) (VIF: 22.07)
제거: G

In [2]:
X_vif

,GC=F_Volume,^VIX_Volatility,GC=F_Volatility,SHY_Volatility,PER(배)
0,3.761200,0.845868,1.791759,0.039222,29.475254
1,3.178054,1.071584,2.292537,0.067658,29.475254
2,6.210600,0.845868,2.721292,0.131028,29.475254
3,5.786897,1.558145,3.763525,0.122215,29.475254
4,5.552960,0.932164,2.240712,0.104361,29.475254
...,...,...,...,...,...
749,0.000000,1.040277,4.127134,0.039222,57.209831
750,6.489205,1.275363,4.234107,0.029558,57.209831
751,6.327937,1.311032,4.000036,0.076963,57.209831
752,6.478510,1.300192,3.353410,0.048793,57.209831


In [3]:
df

,Date,High,Low,Volume,Close,Volatility,DX-Y.NYB_Close,GC=F_Close,GLD_Close,SHY_Close,...,PBR(배),PCR(배),SPS(달러),DPS(달러),EPS(달러),BPS(달러),CPS(달러),ROA(%),ROIC(%),y_target_log
0,2022-11-07,5.070581,5.046110,17.002900,5.064420,1.348107,4.701571,7.424464,5.048894,4.389623,...,2.454816,3.393581,4.010877,0.778391,1.857357,3.640886,2.536491,2.174752,2.881617,5.940040
1,2022-11-08,5.082543,5.056325,17.138536,5.071352,1.428167,4.697020,7.445476,5.071730,4.390615,...,2.454816,3.393581,4.010877,0.778391,1.857357,3.640886,2.536491,2.174752,2.881617,5.945421
2,2022-11-09,5.073485,5.048286,17.073872,5.050772,1.379937,4.705468,7.444307,5.066700,4.392224,...,2.454816,3.393581,4.010877,0.778391,1.857357,3.640886,2.536491,2.174752,2.881617,5.924603
3,2022-11-10,5.108822,5.074949,17.423932,5.103842,1.706791,4.684074,7.467543,5.096691,4.397654,...,2.454816,3.393581,4.010877,0.778391,1.857357,3.640886,2.536491,2.174752,2.881617,5.978101
4,2022-11-11,5.125237,5.089695,17.287892,5.111593,1.770465,4.666171,7.476472,5.103275,4.397408,...,2.454816,3.393581,4.010877,0.778391,1.857357,3.640886,2.536491,2.174752,2.881617,5.987733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,2025-11-03,5.790372,5.762222,16.803182,5.777262,2.206111,4.603869,8.294125,5.910200,4.415461,...,2.695700,3.270226,4.238562,1.063073,2.303466,3.908930,2.449265,2.529518,3.262714,6.526993
750,2025-11-04,5.782241,5.755683,16.885551,5.766972,2.140554,4.607368,8.280888,5.892528,4.415824,...,2.695700,3.270226,4.238562,1.063073,2.303466,3.908930,2.449265,2.529518,3.262714,6.515068
751,2025-11-05,5.784934,5.756235,16.750630,5.774297,2.219747,4.607168,8.289112,5.904026,4.415099,...,2.695700,3.270226,4.238562,1.063073,2.303466,3.908930,2.449265,2.529518,3.262714,6.518528
752,2025-11-06,5.782635,5.753338,16.878282,5.765556,2.237762,4.602467,8.289012,5.902825,4.416307,...,2.695700,3.270226,4.238562,1.063073,2.303466,3.908930,2.449265,2.529518,3.262714,6.507740


In [4]:
# 1) X_vif에 Date 붙이기 (df에서 가져옴)
X_vif_with_date = df[["Date"]].join(X_vif)

# 2) Date 기준으로 y_target 붙이기
merged = X_vif_with_date.merge(
    df[["Date", "y_target_log"]],
    on="Date",
    how="inner"
)

# 3) 컬럼 순서 정리 (원하면)
cols = ["Date"] + [c for c in merged.columns if c not in ["Date", "y_target_log"]] + ["y_target_log"]
merged = merged[cols]

print(merged.head())
print(merged.shape)


         Date  GC=F_Volume  ^VIX_Volatility  GC=F_Volatility  SHY_Volatility  \
0  2022-11-07     3.761200         0.845868         1.791759        0.039222   
1  2022-11-08     3.178054         1.071584         2.292537        0.067658   
2  2022-11-09     6.210600         0.845868         2.721292        0.131028   
3  2022-11-10     5.786897         1.558145         3.763525        0.122215   
4  2022-11-11     5.552960         0.932164         2.240712        0.104361   

      PER(배)  y_target_log  
0  29.475254      5.940040  
1  29.475254      5.945421  
2  29.475254      5.924603  
3  29.475254      5.978101  
4  29.475254      5.987733  
(754, 7)


In [5]:
merged

,Date,GC=F_Volume,^VIX_Volatility,GC=F_Volatility,SHY_Volatility,PER(배),y_target_log
0,2022-11-07,3.761200,0.845868,1.791759,0.039222,29.475254,5.940040
1,2022-11-08,3.178054,1.071584,2.292537,0.067658,29.475254,5.945421
2,2022-11-09,6.210600,0.845868,2.721292,0.131028,29.475254,5.924603
3,2022-11-10,5.786897,1.558145,3.763525,0.122215,29.475254,5.978101
4,2022-11-11,5.552960,0.932164,2.240712,0.104361,29.475254,5.987733
...,...,...,...,...,...,...,...
749,2025-11-03,0.000000,1.040277,4.127134,0.039222,57.209831,6.526993
750,2025-11-04,6.489205,1.275363,4.234107,0.029558,57.209831,6.515068
751,2025-11-05,6.327937,1.311032,4.000036,0.076963,57.209831,6.518528
752,2025-11-06,6.478510,1.300192,3.353410,0.048793,57.209831,6.507740


In [6]:
merged["Date"] = pd.to_datetime(merged["Date"])
merged = merged.set_index("Date")

In [7]:
merged.index

DatetimeIndex(['2022-11-07', '2022-11-08', '2022-11-09', '2022-11-10',
               '2022-11-11', '2022-11-14', '2022-11-15', '2022-11-16',
               '2022-11-17', '2022-11-18',
               ...
               '2025-10-27', '2025-10-28', '2025-10-29', '2025-10-30',
               '2025-10-31', '2025-11-03', '2025-11-04', '2025-11-05',
               '2025-11-06', '2025-11-07'],
              dtype='datetime64[ns]', name='Date', length=754, freq=None)

In [10]:
# 저장
merged.to_csv("../preprocessing/log공선성제거_Final_data.csv", index=True, encoding="utf-8-sig")